## Regression Analysis on the outputs from the previously coded algorithms

- The sampling rate of the empkins dataset 1000 Hz does not match the sampling rate of the guardian dataset 500 Hz. How to deal with that? Convert to milliseconds. Do it when building the train and test data
- Normalize B-Point samples acording to start and end point of heartbeat as part of data preprocessing?
    - Try both approaches
- Start without feature selection since the tree should use all outputs generated by the algorithms
- How to impute nan values? If normalized between 0 and 1 just use mean?
    - drop them first
    - Background: Many algorithms don't handle NaN values
    - Check how many entries contain NaN
    - Use a SimpleImputer with e.g. mean or KNNImputer first to test the pipeline properly (https://scikit-learn.org/1.5/modules/impute.html)
- GroupKFold could be used for cross validation to ensure that a participant is not present in the train and testdata --> until now I only used KFold 
- Splitting of the data:
    - use biopyskit and apply groupkfold. Mutliindex can remain in dataframe its important, that one hearbeat per row
    - I splitted the data on the participant level using GroupShuffleSplit
    - I treated the datasets separately --> 20% test data for guardian and empkins dataset
    - train and test data of empkins and guardian dataset where joined --> Amount of guardian and empkins data in the train and test dataset is approximately equal

### Setup and helper functions

In [2]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

#Classification
from sklearn.tree import DecisionTreeClassifier

# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

# Cross-Validation
from sklearn.model_selection import KFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter

import matplotlib.pyplot as plt

%matplotlib widget
%load_ext autoreload
%autoreload 2

### Load data

In [3]:
data_path = Path("../../results/train_test_data")
data_path

WindowsPath('../../results/train_test_data')

In [4]:
models_path = Path("../../results/models")

In [10]:
train_data = pd.read_csv(data_path.joinpath("train_data.csv")).drop(columns=["Unnamed: 0"])
train_target = pd.read_csv(data_path.joinpath("train_target.csv")).drop(columns=["Unnamed: 0"])

In [4]:
X = train_data.to_numpy()
y = train_target.to_numpy()

### Impute the missing values by using the mean of the row
- the length of a heartbeat can vary
    - Therefore averaging over heartbeats is not a good approach
    - It would be better to average over the rows
        --> Transpose the data before imputation

In [21]:
print(f"Does the target data contain np.nan values?\n {train_target.isna().any()}\n")
print(f"Does the train data contain np.nan values?\n {train_data.isna().any()}")

Does the target data contain np.nan values?
 b_point_samplereference    False
dtype: bool

Does the train data contain np.nan values?
 multiple-conditions_autoregression          False
multiple-conditions_linear-interpolation    False
multiple-conditions_none                     True
second-derivative_autoregression            False
second-derivative_linear-interpolation      False
second-derivative_none                       True
straight-line_autoregression                False
straight-line_linear-interpolation          False
straight-line_none                          False
third-derivative_autoregression             False
third-derivative_linear-interpolation       False
third-derivative_none                        True
dtype: bool


In [6]:
data_imputer = SimpleImputer(strategy="mean")

In [10]:
print(f"Shape of X: {X.shape}")

Shape of X: (9147, 12)


In [14]:
X_train_imputed_transposed = data_imputer.fit_transform(X.T)
print(f"Shape of X.T after data imputation: {X_train_imputed_transposed.shape}")

Shape of X.T after data imputation: (12, 9147)


In [15]:
X_train_imputed = X_train_imputed_transposed.T
print(f"Shape of X after data imputation: {X_train_imputed.shape}")

Shape of X after data imputation: (9147, 12)


In [41]:
X_train_imputed = X_train_imputed.astype(np.float64)
y = y.astype(np.float64)

### Specify Estimator Combinations and Parameters for Hyperparameter Search

In [63]:
model_dict = {
    "scaler": {"StandardScaler": StandardScaler(), "MinMaxScaler": MinMaxScaler()},
    "clf": {
        "KNeighborsRegressor": KNeighborsRegressor(),
        "DecisionTreeRegressor": DecisionTreeRegressor(),
        "SVR": SVR(),
        #"AdaBoostRegressor": AdaBoostRegressor(),
    },
}

In [68]:
params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "KNeighborsRegressor": {"n_neighbors": [2, 4]},
    "DecisionTreeRegressor": {"max_depth": [2, 4]},
    "SVR": [
        {
            "kernel": ["linear"],
            "C": np.logspace(start=-2, stop=2, num=5)
        },
        {
            "kernel": ["rbf"],
            "C": np.logspace(start=-2, stop=2, num=5),
            "gamma": np.logspace(start=-2, stop=2, num=5)
        }
    ],
    #"AdaBoostRegressor": {
    #    "estimator": [DecisionTreeClassifier(max_depth=1)],
    #    "n_estimators": np.arange(20, 110, 10),
    #    "learning_rate": np.arange(0.6, 1.1, 0.1)
    #},
}

In [69]:
hyper_search_dict = {"DecisionTreeRegressor": {"search_method": "random", "n_iter":2}}

### Setup PipelinePermuter and Cross-Validations for Model Evaluation

In [70]:
pipeline_permuter = SklearnPipelinePermuter(
    model_dict=model_dict, param_dict=params_dict, hyper_search_dict=hyper_search_dict
)

In [71]:
outer_cv = KFold(5)
inner_cv = KFold(5)

pipeline_permuter.fit(X=X_train_imputed, y=y, outer_cv=outer_cv, inner_cv=inner_cv, scoring="r2")

Pipeline Combinations:   0%|          | 0/6 [00:00<?, ?it/s]

### Running hyperparameter search for pipeline: (('scaler', 'StandardScaler'), ('clf', 'KNeighborsRegressor')) with 1 parameter grid(s):
Parameter grid #0 ({'search_method': 'grid'}): {'clf__n_neighbors': [2, 4]}


Outer CV:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 2 candidates, totalling 10 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


...

Fitting 5 folds for each of 25 candidates, totalling 125 fits


c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 25 candidates, totalling 125 fits


c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


### Display the resuslts of the pipeline permuter

To print the results I had to exclude the conf matrix in the Biopsykit function.  
Make sure to include it again afterwards

In [72]:
pipeline_permuter.metric_summary()

true_labels  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   
                DecisionTreeRegressor  [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   
                SVR                    [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   
MinMaxScaler    KNeighborsRegressor    [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   
                DecisionTreeRegressor  [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   
                SVR                    [[1074.0], [1849.0], [2518.0], [3252.0], [3933...   

                                                                       true_labels_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   
                DecisionTreeRegressor  [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   
                SVR                    [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   
MinMaxScaler    KNeighborsRegressor    [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   
                DecisionTreeRegressor  [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   
                SVR                    [[[1074.0], [1849.0], [2518.0], [3252.0], [393...   

                                                                        predicted_labels  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [[1114.25], [1855.5], [2533.75], [3255.5], [39...   
                DecisionTreeRegressor  [1122.7484143763213, 1122.7484143763213, 2855....   
                SVR                    [1346.474763842738, 1897.7439746244672, 2502.1...   
MinMaxScaler    KNeighborsRegressor    [[1114.25], [1855.5], [2533.75], [3255.5], [39...   
                DecisionTreeRegressor  [1122.7484143763213, 1122.7484143763213, 2855....   
                SVR                    [1377.9323030828727, 2063.1115307359905, 2706....   

                                                                  predicted_labels_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [[[1114.25], [1855.5], [2533.75], [3255.5], [3...   
                DecisionTreeRegressor  [[1122.7484143763213, 1122.7484143763213, 2855...   
                SVR                    [[1346.474763842738, 1897.7439746244672, 2502....   
MinMaxScaler    KNeighborsRegressor    [[[1114.25], [1855.5], [2533.75], [3255.5], [3...   
                DecisionTreeRegressor  [[1122.7484143763213, 1122.7484143763213, 2855...   
                SVR                    [[1377.9323030828727, 2063.1115307359905, 2706...   

                                                                           train_indices  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   
                DecisionTreeRegressor  [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   
                SVR                    [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   
MinMaxScaler    KNeighborsRegressor    [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   
                DecisionTreeRegressor  [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   
                SVR                    [1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...   

                                                                     train_indices_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  KNeighborsRegressor    [[1830, 1831, 1832, 1833, 1834, 1835, 1836, 18...   
                DecisionTreeRegressor  [[1830, 1831, 1832, 1833, 1834, 1835, 1836, 18...   
                SVR     

### Save the results of the pipeline permuter to a pickle file

In [52]:
pipeline_permuter.to_pickle(models_path.joinpath("DT_SVR_AdaBoost.pkl"))